In [1]:
import chainer
import sklearn
import datetime
import numpy as np
import time
import os
import librosa
import chainer.links as L
import chainer.functions as F
import matplotlib.pylab as plt
import joblib

from chainer import training
from chainer.training import extensions
from sklearn.model_selection import train_test_split
from sklearn import datasets, model_selection, svm, metrics
from chainer import Chain, Variable, optimizer, optimizers, serializers,training
from sklearn.datasets.base import get_data_home 
from sklearn.datasets import fetch_openml
from matplotlib import cm
from chainer import Sequential
from librosa import display
from sklearn.model_selection import train_test_split
print('ok')

ok


In [2]:
class CNN(Chain):
    def __init__(self):
        # クラスの初期化
        super(CNN, self).__init__(            
            conv1 = L.Convolution2D(1, 20, 4), # フィルター3　ボケを作ってる、入力、出力
            conv2 = L.Convolution2D(20, 30, 3), # フィルター4
            conv3 = L.Convolution2D(30, 40, 3),
            conv4 = L.Convolution2D(40, 50, 3),
            #conv5 = L.Convolution2D(40, 50, 3),
            l1 = L.Linear(800, 500),
            l2 = L.Linear(500, 500),
            l3 = L.Linear(500, 10, initialW=np.zeros((10, 500), dtype=np.float32))
        )
    #@staticmethod
    def __call__(self,x):
        # 順伝播の計算を行う関数
        # :param x: 入力値
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.max_pooling_2d(F.relu(self.conv3(h)), 2)
        h = F.relu(self.conv4(h))
        #h = F.relu(self.conv5(h))
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        y = self.l3(h)
        return y

In [3]:
#教師データ（JAPANESE）の読み込み
path = 'datasetjp/'
lst = []

n_mfccs = 63
start_time = time.time()
#教師データを変換
for subdir, dirs, files in os.walk(path):
  for file in files :
      try:
        #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
        X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
        
        mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=n_mfccs)
        
        t = len(mfccs.T)
        start = int((t - n_mfccs)/2)
        end = int((t + n_mfccs)/2)
        
        mfccs=mfccs[:, start:end]
        #print(mfccs)

        # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
        # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
        #print(file)
        file = int(file[0:1]) - 1 
        print(mfccs.shape, file)

        mfccs = np.expand_dims(mfccs, axis=0)
        arr = mfccs,file
        
        #print(arr)
        lst.append(arr)
      # If the file is not valid, skip it
      except ValueError:
        continue
print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 1
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 2
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 3
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 4
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 5
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 6
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 7
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 0
(63, 63) 1

In [4]:
#モデルの定義を行う
model = L.Classifier(CNN())
optimizer = optimizers.Adam()
optimizer.setup(model)

In [5]:
print(lst)

[(array([[[-2.1472501e+02, -2.7365063e+02, -2.3791968e+02, ...,
         -2.3029922e+02, -2.2193518e+02, -2.1630099e+02],
        [ 1.6895428e+02,  1.5832140e+02,  1.5076093e+02, ...,
          1.7051556e+02,  1.6370587e+02,  1.5883842e+02],
        [-3.0199932e+01, -8.4365149e+00, -1.9896759e+01, ...,
         -3.2502087e+01, -4.0952988e+01, -3.4138939e+01],
        ...,
        [ 5.9527388e+00,  5.0305638e+00,  4.0538521e+00, ...,
         -5.2591963e+00, -1.0691929e+01, -8.0395756e+00],
        [-1.7497685e+00,  4.7029867e+00,  4.1197758e+00, ...,
         -5.7849503e+00, -9.4516811e+00, -7.4053307e+00],
        [-2.5953925e-01,  1.7383695e+00,  2.5642850e+00, ...,
         -6.3470650e+00, -7.5656185e+00,  1.4734077e-01]]], dtype=float32), 0), (array([[[-1.8374197e+02, -1.9239079e+02, -2.2896638e+02, ...,
         -3.0359238e+02, -3.0858554e+02, -3.1244293e+02],
        [ 1.8123529e+02,  1.6650818e+02,  1.4165991e+02, ...,
          6.8177597e+01,  6.9539429e+01,  7.1488358e+01],
  

In [6]:
#学習（epocとbatchsizeの設定）
n_epoch = 100
n_batchsize = 50#100

# 学習開始する
print("Train")
train, test = chainer.datasets.split_dataset_random(lst, int(len(lst)*0.7)) # 1440(データセットの個数)-432=1008件を学習用、432件をテスト用

train_iter = chainer.iterators.SerialIterator(train, n_batchsize)
test_iter = chainer.iterators.SerialIterator(test, n_batchsize, repeat=False, shuffle=False)
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (n_epoch, "epoch"))

trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ["epoch", "main/loss", "validation/main/loss", "main/accuracy", "validation/main/accuracy", "elapsed_time"])) # エポック、学習損失、テスト損失、学習正解率、テスト正解率、経過時間
#trainer.extend(extensions.ProgressBar()) # プログレスバー出力
trainer.run()

print("Finished Learning!!")

Train
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           2.12964     1.86274               0.16           0.232727                  4.03554       
2           1.66092     1.66096               0.365714       0.351818                  8.17172       
3           1.33879     1.50281               0.505714       0.444848                  13.3744       
4           1.11143     1.36705               0.616667       0.489394                  18.1653       
5           0.831374    1.19701               0.685714       0.573939                  23.7858       
6           0.628577    1.15381               0.771429       0.618788                  29.1345       
7           0.363391    1.16765               0.88           0.648182                  35.09         
8           0.328563    1.12903               0.9            0.628182                  39.9415       
9           0.206026    1.09903               0.931429       0.665152         

80          2.79699e-06  1.71466               1              0.719394                  501.668       
81          2.64304e-06  1.72134               1              0.726061                  507.464       
82          2.62533e-06  1.72719               1              0.726061                  513.754       
83          2.51611e-06  1.73205               1              0.726061                  519.379       
84          2.46729e-06  1.73261               1              0.726061                  525.326       
85          2.4019e-06  1.73644               1              0.719394                  532.154       
86          2.12193e-06  1.73824               1              0.719394                  537.682       
87          2.1771e-06  1.74365               1              0.719394                  544.326       
88          2.12261e-06  1.75028               1              0.711818                  550.996       
89          2.0068e-06  1.7535                1              0.719394      

In [7]:
def predict(model, x):
    y = np.argmax(model.predictor(x=np.array([x], dtype="float32")).data)
    return y

turu_total=0
predict_total=0
#idx = np.random.choice(test, test)
print(len(test))
print(len(lst))
for i in range(len(test)):
    y_=predict(model, test[i][0])
    y =test[i][1]
    print(y_)
    #predict_total+=1
    if y==y_:
        turu_total+=1
print(turu_total*100/len(test),'%')

144
480
0
7
2
1
5
3
1
0
4
4
5
7
1
4
5
1
5
1
0
0
2
2
5
4
1
6
5
4
1
0
3
4
2
3
1
3
1
0
2
3
5
0
2
0
6
2
3
0
6
2
0
7
5
5
5
1
4
5
4
0
3
5
5
1
5
7
4
7
1
0
1
1
1
6
2
6
5
5
5
6
0
5
1
5
2
7
4
0
4
7
6
6
3
1
1
3
1
3
6
6
1
7
1
7
2
2
6
0
4
3
2
0
2
0
4
1
0
2
6
6
4
2
6
4
7
5
4
0
2
1
6
6
5
1
6
4
2
1
3
0
6
0
7
5
71.52777777777777 %
